In [6]:
import requests
import time
from datetime import datetime
def get_market_by_slug(slug: str) -> dict:
    """Fetch market details directly by slug (no searching)"""
    response = requests.get(
        f"https://gamma-api.polymarket.com/markets",
        params={"slug": slug}
    )
    markets = response.json()
    return markets[0] if markets else None
def get_btc_15m_market(timestamp: int) -> dict:
    """Get BTC 15-min market for a specific timestamp"""
    slug = f"btc-updown-15m-{timestamp}"
    return get_market_by_slug(slug)
def get_next_market_timestamp(current_ts: int) -> int:
    """Calculate next 15-min market timestamp"""
    return current_ts + 900
# Example: Get current market
current_ts = 1766300400
market = get_btc_15m_market(current_ts)
if market:
    print(f"Question: {market['question']}")
    print(f"End Date: {market['endDate']}")
    print(f"Condition ID: {market['conditionId']}")
    print(f"Token IDs: {market['clobTokenIds']}")  # This is what WebSocket needs

    yes_token_id = market['clobTokenIds'][0]
    no_token_id = market['clobTokenIds'][1]

Question: Bitcoin Up or Down - December 21, 2:00AM-2:15AM ET
End Date: 2025-12-21T07:15:00Z
Condition ID: 0xf809c552f95cc273f6eae8798d4f3ccc55e862d862dc76294d088189c2872312
Token IDs: ["5425927090557208584672104105719205419160760668441402881579391176347511483878", "19238554207893600170012493978895250770634787854012398597180064156204748273513"]


In [2]:
import requests
import time
from datetime import datetime
def get_market_by_slug(slug: str) -> dict:
    """Fetch market details directly by slug (no searching)"""
    response = requests.get(
        f"https://gamma-api.polymarket.com/markets",
        params={"slug": slug}
    )
    markets = response.json()
    return markets[0] if markets else None
def get_btc_15m_market(timestamp: int) -> dict:
    """Get BTC 15-min market for a specific timestamp"""
    slug = f"btc-updown-15m-{timestamp}"
    return get_market_by_slug(slug)
def get_next_market_timestamp(current_ts: int) -> int:
    """Calculate next 15-min market timestamp"""
    return current_ts + 900
# Example: Get current market
current_ts = 1766302200
market = get_btc_15m_market(current_ts)
if market:
    print(f"Question: {market['question']}")
    print(f"End Date: {market['endDate']}")
    print(f"Condition ID: {market['conditionId']}")
    print(f"Token IDs: {market['clobTokenIds']}")  # This is what WebSocket needs

    yes_token_id = market['clobTokenIds'][0]
    no_token_id = market['clobTokenIds'][1]

Question: Bitcoin Up or Down - December 21, 2:30AM-2:45AM ET
End Date: 2025-12-21T07:45:00Z
Condition ID: 0x1ba29e65bb17e67610f515ce637c8c26f15e9f8dc9a14fe56f2cd45bfc368062
Token IDs: ["65746565293819997775300173478269020460544951341535209461919854133553859025488", "42751939750927982695925613318616870696027411162171797959002334921720792035718"]


In [4]:
import json
import threading
import pandas as pd
from websocket import WebSocketApp
from datetime import datetime
import time

# ============ YOUR TOKEN ID ============
TOKEN_ID = "65746565293819997775300173478269020460544951341535209461919854133553859025488"

# ============ STORAGE ============
bids = []
asks = []
snapshots = []  # ADD THIS

# ============ WEBSOCKET ============
def on_open(ws):
    print("Connected!")
    ws.send(json.dumps({"assets_ids": [TOKEN_ID], "type": "market"}))

def on_message(ws, message):
    global bids, asks
    if message == "PONG":
        return
    
    data = json.loads(message)
    events = data if isinstance(data, list) else [data]
    
    for event in events:
        if event.get("event_type") == "book" and event.get("asset_id") == TOKEN_ID:
            bids = event.get("bids", [])
            asks = event.get("asks", [])

# ============ SAMPLER (2x per second) ============  # ADD THIS BLOCK
def sampler():
    while True:
        if bids and asks:
            snapshots.append({
                "timestamp": datetime.now(),
                "best_bid": float(bids[-1]["price"]),
                "best_ask": float(asks[-1]["price"]),
                "bid_size": float(bids[-1]["size"]),
                "ask_size": float(asks[-1]["size"]),
                "spread": float(asks[-1]["price"]) - float(bids[-1]["price"])
            })
        time.sleep(0.5)

threading.Thread(target=sampler, daemon=True).start()  # ADD THIS

ws = WebSocketApp(
    "wss://ws-subscriptions-clob.polymarket.com/ws/market",
    on_open=on_open,
    on_message=on_message
)
threading.Thread(target=ws.run_forever, daemon=True).start()

In [6]:
df = pd.DataFrame(snapshots)
print(f"Total snapshots: {len(df)}")
display(df.tail(10))

Total snapshots: 23


,timestamp,best_bid,best_ask,bid_size,ask_size,spread
13,2025-12-20 23:37:58.120337,0.46,0.48,49.51,19.58,0.02
14,2025-12-20 23:37:58.622759,0.46,0.48,49.51,19.58,0.02
15,2025-12-20 23:37:59.127877,0.46,0.48,49.51,19.58,0.02
16,2025-12-20 23:37:59.632827,0.46,0.48,49.51,19.58,0.02
17,2025-12-20 23:38:00.133092,0.46,0.48,49.51,19.58,0.02
18,2025-12-20 23:38:00.636203,0.46,0.48,49.51,19.58,0.02
19,2025-12-20 23:38:01.141156,0.46,0.47,108.12,10.00,0.01
20,2025-12-20 23:38:01.644275,0.47,0.48,14.00,45.58,0.01
21,2025-12-20 23:38:02.147712,0.45,0.48,351.79,1045.58,0.03
22,2025-12-20 23:38:02.652011,0.45,0.46,292.90,1825.30,0.01


In [8]:
df = pd.DataFrame(snapshots)
print(f"Total snapshots: {len(df)}")
display(df.tail(10))

Total snapshots: 39


,timestamp,best_bid,best_ask,bid_size,ask_size,spread
29,2025-12-20 23:38:06.181180,0.44,0.46,136.21,1915.3,0.02
30,2025-12-20 23:38:06.685424,0.44,0.46,136.21,1915.3,0.02
31,2025-12-20 23:38:07.190569,0.44,0.46,136.21,1915.3,0.02
32,2025-12-20 23:38:07.692582,0.44,0.46,136.21,1915.3,0.02
33,2025-12-20 23:38:08.193868,0.44,0.46,136.21,1915.3,0.02
34,2025-12-20 23:38:08.697606,0.43,0.46,413.43,119.0,0.03
35,2025-12-20 23:38:09.201685,0.43,0.46,496.35,109.0,0.03
36,2025-12-20 23:38:09.706788,0.43,0.46,470.82,104.0,0.03
37,2025-12-20 23:38:10.211379,0.43,0.46,430.82,104.0,0.03
38,2025-12-20 23:38:10.714509,0.43,0.46,430.82,104.0,0.03


In [10]:
df = pd.DataFrame(snapshots)
print(f"Total snapshots: {len(df)}")
display(df.tail(10))

Total snapshots: 105


,timestamp,best_bid,best_ask,bid_size,ask_size,spread
95,2025-12-20 23:38:39.416035,0.46,0.47,588.92,1674.10,0.01
96,2025-12-20 23:38:39.916453,0.46,0.47,635.92,1669.10,0.01
97,2025-12-20 23:38:40.421507,0.46,0.47,635.92,1650.10,0.01
98,2025-12-20 23:38:40.924916,0.46,0.47,635.92,1650.10,0.01
99,2025-12-20 23:38:41.426648,0.46,0.47,635.92,1650.10,0.01
100,2025-12-20 23:38:41.929341,0.47,0.50,636.00,892.19,0.03
101,2025-12-20 23:38:42.434440,0.47,0.50,23.00,710.63,0.03
102,2025-12-20 23:38:42.939013,0.46,0.47,233.92,1635.10,0.01
103,2025-12-20 23:38:43.443930,0.46,0.47,293.92,1432.82,0.01
104,2025-12-20 23:38:43.948244,0.46,0.47,277.66,1391.16,0.01


In [1]:
!pwd

/Users/ryanjain


In [ ]:
ws.c